<a href="https://colab.research.google.com/github/Marlburo/colab_nb/blob/Keras/colab_nb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import os
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt


In [3]:
#housing_test = pd.read_csv("/content/sample_data/california_housing_test.csv")
#housing_train = pd.read_csv("/content/sample_data/california_housing_train.csv")
mnist_test = pd.read_csv("/content/sample_data/mnist_test.csv")
mnist_train = pd.read_csv("/content/sample_data/mnist_train_small.csv")

In [4]:
#housing_test.describe()
#mnist_test
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name = 'predictions')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()#mnist_train, mnist_test

#preprocess the data
x_train = x_train.reshape(60000, 784).astype("float32")/255
x_test = x_test.reshape(10000, 784).astype("float32")/255

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

#reserve 10000 samples for validation
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    #loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    #list of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

In [11]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=4,
    #We pass some validation for
    #monitoring validation loss and metrics
    #at the end of each epoch
    validation_data=(x_val, y_val),
)

Fit model on training data
Epoch 1/4
782/782 [==============================] - 2s 3ms/step - loss: 3.1342e-04 - sparse_categorical_accuracy: 0.9998 - val_loss: 0.4201 - val_sparse_categorical_accuracy: 0.9745
Epoch 2/4
782/782 [==============================] - 2s 2ms/step - loss: 5.4167e-04 - sparse_categorical_accuracy: 0.9998 - val_loss: 0.4264 - val_sparse_categorical_accuracy: 0.9738
Epoch 3/4
782/782 [==============================] - 2s 2ms/step - loss: 2.5385e-04 - sparse_categorical_accuracy: 0.9999 - val_loss: 0.4277 - val_sparse_categorical_accuracy: 0.9745
Epoch 4/4
782/782 [==============================] - 2s 3ms/step - loss: 7.1716e-04 - sparse_categorical_accuracy: 0.9998 - val_loss: 0.4451 - val_sparse_categorical_accuracy: 0.9749


In [12]:
history.history

{'loss': [0.00031341525027528405,
  0.0005416739149950445,
  0.00025385365006513894,
  0.0007171611068770289],
 'sparse_categorical_accuracy': [0.9998400211334229,
  0.9998199939727783,
  0.9998999834060669,
  0.9998400211334229],
 'val_loss': [0.42014598846435547,
  0.4263850450515747,
  0.4276760220527649,
  0.44508275389671326],
 'val_sparse_categorical_accuracy': [0.9745000004768372,
  0.973800003528595,
  0.9745000004768372,
  0.9749000072479248]}

In [13]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print ("test loss, test acc", results)

#generate predictions (probabilities -- the output of the last layer
#on new data using predict
print ("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
79/79 [==============================] - 0s 2ms/step - loss: 0.4043 - sparse_categorical_accuracy: 0.9751
test loss, test acc [0.40428951382637024, 0.9750999808311462]
Generate predictions for 3 samples
predictions shape: (3, 10)


In [14]:
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

In [15]:
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)

In [16]:
def get_uncompiled_model():
  inputs = keras.Input(shape=(784,), name="digits")
  x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
  x = layers.Dense(64, activation="relu", name="dense_2")(x)
  outputs = layers.Dense(10, activation="softmax",name="predictions")(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model

def get_compiled_model():
  model = get_uncompiled_model()
  model.compile(
      optimizer="rmsprop",
      loss="sparse_categorical_crossentropy",
      metrics=["sparse_categorical_accuracy"],
  )
  return model